In [1]:
from source import neighbor_based_recommender

In [2]:
recommender = neighbor_based_recommender(100000)

In [3]:
recommender.train(neighbor_size = 5, recalculate = False)

In [ ]:
recommender.runtime

In [4]:
rmse_result,user_rmse,movie_rmse=recommender.RMSE()

In [ ]:
top_k_precision, _ = recommender.top_k_precision(k=5)